In [1]:
import pandas as pd
import numpy as np
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from helpers import *

In [80]:
def greedy_regret_repair(path, D, costs, weights=0.5):
    target_length = math.ceil(len(D) / 2)
    path = list(path) + [path[0]]
    edges = []
    nodes_available = [x for x in range(len(D)) if x not in path]
    for idx, node in enumerate(path):
        if idx == len(path)-1: break
        edges.append([node, path[(idx+1) % len(path)]])
    while len(path) < target_length+1 and len(nodes_available) > 0:
        M = np.zeros((len(nodes_available), len(edges)))
        indices = np.array(nodes_available, dtype=int)
        for edge_ix in range(len(edges)):
            a, b = edges[edge_ix]
            var = D[a, :] + D[:, b] - D[a, b] + costs
            M[:,edge_ix] = var[indices]
        best_score = -MAX_DIST
        replaced_edge = 0
        best_node = 0
        for node_idx in range(len(nodes_available)):
            best, second_best = np.partition(M[node_idx], 1)[:2]
            regret = second_best - best
            score = weights * regret - (1 - weights) * np.min(M[node_idx])
            if score > best_score:
                best_score = score
                replaced_edge = np.argmin(M[node_idx])
                best_node = nodes_available[node_idx]
        path.insert(replaced_edge+1, best_node)
        nodes_available.remove(best_node)
        a, b = edges[replaced_edge]
        edges.pop(replaced_edge)
        edges.insert(replaced_edge, [a, best_node])
        edges.insert(replaced_edge + 1, [best_node, b])
    return path[:-1]

def get_edges(path):
    edges = []
    for idx, node in enumerate(path):
        edge = [node, path[(idx+1) % len(path)]]
        edges.append(edge)
    return edges

def get_common_edges(path1, path2):
    common_edges = []
    for idx, edge_start in enumerate(path1):
        edge_end = path1[(idx+1) % len(path1)]
        if edge_start in path2:
            e_idx = path2.index(edge_start)
            if edge_end == path2[(e_idx+1) % len(path2)]:
                edge = [edge_start, edge_end]
                common_edges.append(edge)
            elif edge_end == path2[(e_idx-1) % len(path2)]:
                edge = [edge_start, edge_end]
                common_edges.append(edge)
    return common_edges


def get_initial_edges_path(path1, path2):
    offspring = []
    edges = get_edges(path1)
    common_edges = get_common_edges(path1, path2)
    for edge in edges[1:]:
        if edge in common_edges or edge[::-1] in common_edges:
            if len(offspring) > 0:
                if offspring[-1] != edge[0]:
                    offspring.append(edge[0])
                if offspring[0] != edge[1]:
                    offspring.append(edge[1])
            else:
                offspring.extend(edge)
    return offspring

def combine_random(path1, path2, D):
    common_nodes = [node for node in path1 if node in path2]
    offspring = get_initial_edges_path(path1, path2)
    nodes_available = [x for x in common_nodes if x not in offspring]
    offspring.extend(nodes_available)
    nodes_available = [x for x in range(len(D)) if x not in offspring]
    while len(offspring) < len(path1):
        node = random.choice(nodes_available)
        offspring.append(node)
        nodes_available.remove(node)
    return offspring

def combine_heuristic(path1, path2, D, costs):
    offspring = get_initial_edges_path(path1, path2)
    if len(offspring) == 0:
        offspring.extend([path1[0], path1[1]])
    repaired = greedy_regret_repair(offspring, D, costs)
    return repaired

# TODO change the implementation so that you can pass nodes available to regret repair

def HEA(D, costs, LS_enable=False,  POP_SIZE = 20, time_limit=200):
    counter = 0
    population = []
    scores = []
    while len(population) < POP_SIZE:
        path = random_sequence(D)
        path = list(local_search(path, costs, D))
        score = evaluate(D, path, costs)
        if score not in scores:
            population.append(list(path))
            scores.append(score)
    time_start = time.time()
    while True:
        counter += 1
        parents = random.choices(population, k=2)
        if random.random() < 0.5:
            offspring = combine_random(parents[0], parents[1], D)
            score = evaluate(D, offspring, costs)
        else:
            offspring = combine_heuristic(parents[0], parents[1], D, costs)
            score = evaluate(D, offspring, costs)
        if LS_enable:
            offspring = list(local_search(offspring, costs, D))
        offspring_score = evaluate(D, offspring, costs)
        worst_idx = scores.index(max(scores))
        if offspring_score < scores[worst_idx]:
            if offspring not in population and offspring_score not in scores:
                population[worst_idx] = offspring
                scores[worst_idx] = offspring_score

        if time.time() - time_start > time_limit:
            break
    best_found = population[scores.index(min(scores))]
    return best_found, counter

In [ ]:
def evaluate_solution(filename, D, costs, LS_enable=False,  POP_SIZE = 20, time_limit=200):
    start = time.time()
    path, counter = HEA(D, costs, LS_enable=False,  POP_SIZE=POP_SIZE, time_limit=time_limit)
    end = time.time()
    score = evaluate(D, path, costs)
    return {
        'Filename': filename,
        'Version': 'LS' if LS_enable else 'no LS',
        'Iterations': counter,
        'Path': path,
        'Score': score,
        'Time': (end - start)
    }

# %%
# result = evaluate_solution('TSPA.csv', D, costs)
# result

# %%
if __name__ == "__main__":
    test_start = time.time()
    files = ['TSPA.csv', 'TSPB.csv', 'TSPC.csv', 'TSPD.csv']
    for filename in files:
        prefix = "HEA"
        mapping_out_files= {
            'TSPA.csv' : f'{prefix}_TSPA_out.csv',
            'TSPB.csv' : f'{prefix}_TSPB_out.csv',
            'TSPC.csv' : f'{prefix}_TSPC_out.csv',
            'TSPD.csv' : f'{prefix}_TSPD_out.csv'
        }
        nodes, costs, D = read_data(filename)
        num_iterations = 10
        time_limit = 200
        results = Parallel(n_jobs=-1)(delayed(evaluate_solution)(filename, D, costs, LS_enable=LS_enable, POP_SIZE=20, time_limit=time_limit)
                                    for LS_enable in [True, False]
                                    for i in range(num_iterations)
                                    )

        results_df = pd.DataFrame(results)
        results_df.to_csv(mapping_out_files[filename])
        test_end = time.time()
        print(test_end - test_start)